# How to implment reusing variables
`tf.variable_scope`의 reuse argument를 이용하여, reusing variable (etc. parameter sharing)을 하는 방법을 정리

### Setup

In [1]:
import tensorflow as tf
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

### Template 1
scope를 나누어서 쓸 때, reusing variables를 하는 경우

In [2]:
with tf.variable_scope('test', reuse = False):
    w1 = tf.get_variable(name = 'weight_1',
                     shape = [2,], dtype = tf.float32,
                     initializer = tf.ones_initializer())
    w2 = tf.get_variable(name = 'weight_2', initializer = tf.constant([2.,2.], dtype = tf.float32))

with tf.variable_scope('test', reuse = True):
    w3 = tf.get_variable(name = 'weight_1')

In [3]:
with tf.Session(config = sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    print('w1 : {}'.format(sess.run(w1)))
    print('w2 : {}'.format(sess.run(w2)))
    print('w3 : {}'.format(sess.run(w3)))

w1 : [1. 1.]
w2 : [2. 2.]
w3 : [1. 1.]


In [4]:
# weight_1을 reuse 했으므로, variable list에도 두 개가 나타남
tf.global_variables()

[<tf.Variable 'test/weight_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'test/weight_2:0' shape=(2,) dtype=float32_ref>]

### Template 2
하나의 scope에서 reusing variables를 하는 경우, `reuse_variables` method를 사용

In [5]:
tf.reset_default_graph()

In [6]:
with tf.variable_scope('test') as scope:
    w1 = tf.get_variable(name = 'weight_1',
                     shape = [2,], dtype = tf.float32,
                     initializer = tf.ones_initializer())
    w2 = tf.get_variable(name = 'weight_2', initializer = tf.constant([2.,2.], dtype = tf.float32))
    scope.reuse_variables() 
    w3 = tf.get_variable(name = 'weight_1')

In [7]:
with tf.Session(config = sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    print('w1 : {}'.format(sess.run(w1)))
    print('w2 : {}'.format(sess.run(w2)))
    print('w3 : {}'.format(sess.run(w3)))

w1 : [1. 1.]
w2 : [2. 2.]
w3 : [1. 1.]


In [8]:
# weight_1을 reuse 했으므로, variable list에도 두 개가 나타남
tf.global_variables()

[<tf.Variable 'test/weight_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'test/weight_2:0' shape=(2,) dtype=float32_ref>]

### Template 3
하나의 scope에서 reusing variables를 하는 경우, reuse argument에 `tf.AUTO_REUSE` 전달

* `tf.AUTO_REUSE`를 활용하면, ***이미 해당하는 variable name이 있는 경우 그 variable의 값을 가져오고, 아닐 경우 새로운 variable을 생성함***
* ***이 방식으로 활용하는 것을 추천!!***

In [9]:
tf.reset_default_graph()

In [10]:
with tf.variable_scope('test', reuse = tf.AUTO_REUSE):
    w1 = tf.get_variable(name = 'weight_1',
                     shape = [2,], dtype = tf.float32,
                     initializer = tf.ones_initializer())
    w2 = tf.get_variable(name = 'weight_2', initializer = tf.constant([2.,2.], dtype = tf.float32))
    w3 = tf.get_variable(name = 'weight_1')

In [11]:
with tf.Session(config = sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    print('w1 : {}'.format(sess.run(w1)))
    print('w2 : {}'.format(sess.run(w2)))
    print('w3 : {}'.format(sess.run(w3)))

w1 : [1. 1.]
w2 : [2. 2.]
w3 : [1. 1.]


In [12]:
# weight_1을 reuse 했으므로, variable list에도 두 개가 나타남
tf.global_variables()

[<tf.Variable 'test/weight_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'test/weight_2:0' shape=(2,) dtype=float32_ref>]